# 다층 퍼셉트론 기본 구조

## 불량 철판 판별 신경망

In [1]:
!wget -O faults.txt https://archive.ics.uci.edu/ml/machine-learning-databases/00198/Faults.NNA

--2021-02-05 05:09:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00198/Faults.NNA
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299482 (292K) [application/x-httpd-php]
Saving to: ‘faults.txt’

faults.txt          100%[===================>] 292.46K  1.04MB/s    in 0.3s    

2021-02-05 05:09:44 (1.04 MB/s) - ‘faults.txt’ saved [299482/299482]



### 파이썬 모듈 불러들이기

In [2]:
import numpy as np
import time

np.random.seed(1234)

def randomize():
    np.random.seed(time.time())

### 하이퍼파라미터값의 정의

In [3]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

### 메인 함수 정의

In [4]:
def steel_exec(epoch_count=10, mb_size=10, report=1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재 함수 정의

In [5]:
def load_steel_dataset():
    with open('faults.txt', 'r') as f:
        rows = []
        lines = f.readlines()
        for line in lines:
            row = [np.float32(value.strip()) for value in line.split('\t')]
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype='float32')

### 은닉 계층 하나를 위한 파라미터 생성 함수 정의

In [6]:
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

### 학습 및 평가 함수 정의

In [7]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

### 학습 및 평가  데이터 획득 함수 정의

In [8]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

### 학습 실행 함수와 평가 실행 함수 정의

In [9]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

### 은닉 계층 하나를 위한 순전파 함수 정의

In [10]:
def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0)

### 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [11]:
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) 
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

### 소프트맥스 관련 함수 정의

In [12]:
def softmax(x):
    max_elem = np.max(x, axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n,i]
    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

### 은닉 계층 하나를 위한 역전파 함수 정의

In [13]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux

    g_output_w_out = hidden.transpose()                      
    G_w_out = np.matmul(g_output_w_out, G_output)            
    G_b_out = np.sum(G_output, axis=0)                       

    g_output_hidden = pm_output['w'].transpose()             
    G_hidden = np.matmul(G_output, g_output_hidden)          

    pm_output['w'] -= LEARNING_RATE * G_w_out                
    pm_output['b'] -= LEARNING_RATE * G_b_out                
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()                           
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)            
    G_b_hid = np.sum(G_hidden, axis=0)                       
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid                
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid                
    
def relu_derv(y):
    return np.sign(y)

### 가변적 은닉 계층 구성을 위한 파라미터 생성 함수 정의

In [14]:
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

### 가변적 은닉 계층 구성을 위한 순전파 함수 정의

In [15]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

### 가변적 은닉 계층 구성을 위한 역전파 함수 정의

In [16]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)

    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

### 스위치 함수 정의

In [17]:
global hidden_config

def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden1()
    
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)

### 은닉 계층 구조 지정 함수 정의

In [18]:
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info

### 정확도 계산 함수의 재정의

In [19]:
def eval_accuracy(output, y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

## 실행하기

### 단층 퍼셉트론 처리하기

In [23]:
set_hidden([])
steel_exec()

은닉 계층 0개를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 1: loss=16.163, accuracy=0.298/0.389
Epoch 2: loss=15.687, accuracy=0.319/0.248
Epoch 3: loss=15.450, accuracy=0.329/0.317
Epoch 4: loss=15.821, accuracy=0.313/0.192
Epoch 5: loss=15.182, accuracy=0.341/0.194
Epoch 6: loss=15.152, accuracy=0.342/0.394
Epoch 7: loss=15.776, accuracy=0.315/0.363
Epoch 8: loss=15.717, accuracy=0.317/0.187
Epoch 9: loss=15.613, accuracy=0.322/0.212
Epoch 10: loss=15.494, accuracy=0.327/0.407

Final Test: final accuracy = 0.407


### 은닉 계층 하나로 풀어보는 철판의 분량 판별 문제

In [20]:
set_hidden(10)
steel_exec(epoch_count=50, report=10)

은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 10: loss=1.849, accuracy=0.352/0.325
Epoch 20: loss=1.786, accuracy=0.352/0.325
Epoch 30: loss=1.747, accuracy=0.352/0.325
Epoch 40: loss=1.723, accuracy=0.352/0.325
Epoch 50: loss=1.708, accuracy=0.352/0.325

Final Test: final accuracy = 0.325


### 은닉 계층 세 개로 풀어보는 철판의 불량 판별 문제

In [21]:
set_hidden([12, 6, 4])
steel_exec(epoch_count=50, report=10)

은닉 계층 3개를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 10: loss=1.855, accuracy=0.346/0.350
Epoch 20: loss=1.795, accuracy=0.346/0.350
Epoch 30: loss=1.758, accuracy=0.346/0.350
Epoch 40: loss=1.735, accuracy=0.346/0.350
Epoch 50: loss=1.721, accuracy=0.346/0.350

Final Test: final accuracy = 0.350


### 학습률을 바꾸어 풀어보는 철판의 불량 판별 문제

In [22]:
LEARNING_RATE=0.0001
hidden_config = [12, 6, 4]
steel_exec(epoch_count=50, report=10)

은닉 계층 3개를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 10: loss=1.738, accuracy=0.257/0.182
Epoch 20: loss=1.618, accuracy=0.315/0.468
Epoch 30: loss=1.604, accuracy=0.390/0.389
Epoch 40: loss=1.594, accuracy=0.422/0.471
Epoch 50: loss=1.584, accuracy=0.444/0.471

Final Test: final accuracy = 0.471
